<a href="https://colab.research.google.com/github/KorStats/skku/blob/main/code/(NSMC)_Back_transalation_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'  'My Drive'   NSMC_0.02.pkl   NSMC_0.5.pkl   이전   졸업논문


In [ ]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/졸업논문/DATA/NSMC_0.02_증분대상100개.csv', encoding='utf-8-sig')

In [ ]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'id', 'document', 'label'], dtype='object')

# Back-translation

In [ ]:
pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=302e6843878bc3eacaed1cfe3a5044896269afe61945f5ca84625f444a157e4a
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperfram

In [ ]:
import pandas as pd
from googletrans import Translator

def back_translate(text, source_lang, pivot_lang):
  try:
    translator = Translator()
    translated_to_pivot = translator.translate(text, src=source_lang, dest=pivot_lang).text
    back_translated = translator.translate(translated_to_pivot, src=pivot_lang, dest=source_lang).text
    return back_translated
  except:
    return text


def augment_data(texts, source_lang, pivot_langs, multiplier):
    augmented_texts = []
    for text in texts:
        for _ in range(multiplier - 1):
            for pivot_lang in pivot_langs:
                augmented_texts.append(back_translate(text, source_lang, pivot_lang))
    return augmented_texts

def augment_dataframe(df, text_column, category_column, target_category, source_lang, pivot_langs, multiplier):
    df_augmented = df.copy()
    target_texts = df[df[category_column] == target_category][text_column].tolist()
    augmented_texts = augment_data(target_texts, source_lang, pivot_langs, multiplier)
    augmented_rows = [{text_column: text, category_column: target_category} for text in augmented_texts]
    df_augmented = pd.concat([df_augmented, pd.DataFrame(augmented_rows)], ignore_index=True)
    return df_augmented

# 증분하기

###### df_2를 증분

In [ ]:
df_2=df
df_2.shape

(100, 5)

In [ ]:
df_2.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'id', 'document', 'label'], dtype='object')

In [ ]:
df_2['document'].isnull().sum()

0

In [ ]:
df=df_2
#'en', 'ja', 'zh-cn', 'zh-tw', 'fr', 'de', 'es', 'ru', 'pt', 'hi'
#원데이터 + multiplier-1 * pivot_languages 수
pivot_languages = ['en', 'ja', 'zh-cn', 'fr', 'de', 'es', 'ru', 'pt', 'hi', 'it', 'ar', 'id', 'af', 'sq', 'am', 'hy', 'az', 'be', 'bn', 'bg', 'ca', 'cdb', 'ny', 'co', 'hr', 'cs', 'nl', 'eo', 'tl', 'fy', 'gl', 'ha', 'haw', 'hmn', 'is', 'ig', 'ga', 'jw', 'kn', 'kk', 'lo', 'la', 'mk', 'mn', 'ne', 'or', 'pl', 'ro', 'sk'] #20개 언어

In [ ]:
len(pivot_languages)

49

In [ ]:
multiplier=2
df_2_augmented = augment_dataframe(df_2, text_column='document', category_column='label', target_category=0, source_lang='ko', pivot_langs=pivot_languages, multiplier=multiplier)

In [ ]:
df_2_augmented.shape

In [ ]:
df_2_augmented.to_csv('NSMC_불균형_0.02_aug_bt.csv', encoding='utf-8-sig')

In [ ]:
 df_2.shape

# 증분데이터 저장하기

In [ ]:
df_aug=pd.concat([df_1, df_2_augmented])

In [ ]:
df_aug['sentence_label'].value_counts()

,count
sentence_label,
unsafe,8
safe,3


In [ ]:
df_aug.to_csv('증분데이터.csv', encoding='utf-8-sig')